Median and Average Hourly Wages in the USA (1973-2022)


**Introduction:**
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal

Clearly state the question you will try to answer with your project

Identify and describe the dataset that will be used to answer the question

**Preliminary exploratory data analysis:**

Demonstrate that the dataset can be read from the web into R 

Clean and wrangle your data into a tidy format

Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [1]:
library(tidyverse)

read_csv("https://raw.githubusercontent.com/asnbl/dsci-100-2023-24W-group-16/main/median_average_wages.csv")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Rows: 50 Columns: 31
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (31): year, median, average, men_median, men_average, women_median, wome...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,median,average,men_median,men_average,women_median,women_average,white_median,white_average,black_median,⋯,black_women_median,black_women_average,hispanic_women_median,hispanic_women_average,recent_high-school_graduate,men_recent_high-school_graduate,women_recent_high-school_graduate,recent_college_graduate,men_recent_college_graduate,women_recent_college_graduate
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022,22.88,32.00,24.91,35.82,20.74,27.86,24.96,34.49,19.60,⋯,19.00,23.99,17.54,22.60,16.02,16.86,14.77,25.68,28.24,23.74
2021,23.05,32.08,24.93,35.61,21.46,28.28,25.40,34.50,19.45,⋯,18.85,24.13,17.77,22.90,15.23,15.86,14.33,25.68,28.66,23.47
2020,23.64,32.54,25.95,36.03,21.72,28.75,25.98,34.86,19.85,⋯,19.26,24.87,17.55,23.03,14.65,15.48,13.55,27.42,29.01,26.26
2019,22.12,30.36,24.04,33.65,20.42,26.83,24.39,32.79,18.45,⋯,18.08,23.17,17.08,21.50,14.54,15.36,13.47,24.70,26.59,23.30
2018,21.90,29.83,23.42,33.19,19.73,26.21,23.97,32.44,17.57,⋯,17.34,22.55,16.24,20.60,13.83,14.59,12.71,24.08,25.66,22.84
2017,21.55,29.21,23.78,32.27,19.69,25.91,23.87,31.64,17.84,⋯,17.22,21.81,15.75,20.54,13.50,14.23,12.55,24.08,25.94,22.63
2016,21.36,29.00,23.44,32.18,19.44,25.58,23.56,31.30,18.15,⋯,17.13,21.86,15.72,20.18,13.22,13.79,12.44,22.89,24.96,21.36
2015,20.99,28.43,23.19,31.52,19.07,25.11,23.37,30.75,17.38,⋯,16.64,21.65,14.93,19.74,13.19,13.65,12.51,22.75,25.53,20.56
2014,20.65,27.36,22.43,30.08,18.63,24.45,22.73,29.49,17.25,⋯,16.32,21.46,14.83,19.17,12.52,13.39,11.34,22.07,24.89,19.84


**Methods:**

Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?

Describe at least one way that you will visualize the results


**Expected outcomes and significance:**

What do you expect to find?

What impact could such findings have?

What future questions could this lead to?